# Annual Charitable Giving Rates among U.S. Foundations - A Guidestar API Demo

In the U.S. philanthropic realm, there is a well-known tax rule applied to foundations known as the "Five Percent Payout Rule." [The Council on Foundations describes offers the following definition](https://www.ncfp.org/wp-content/uploads/2018/09/The-Five-Percent-Minimum-Payout-Requirement-COF-2000-the-five-percent-minimum-payout-requirement.pdf):

> Each year every private foundation must make eligible charitable expenditures that equal or exceed approximately 5
> percent of the value of its endowment. The word "payout" while convenient is somewhat misleading and is not used in the   > Tax Code section that creates the rule. The word "payout" suggests grants or contributions paid out to other charities.  > Although these grants normally make up more than 93 percent of the expenditures of most foundations, many other expenses  > can also qualify in meeting the minimum payout requirement. In short, the 5 percent payout rule need not be satisfied
> solely with grants.

The program below uses an [API from Guidestar](https://apiportal.guidestar.org/api-static-documentation-v2)--a free charity watchdog service that is widely used by funders and charities alike--to parse a given foundation's tax forms (Form 990) to determine what percentage of a foundation's assets was donated to charity in their most recently reported tax year. 

As explained in the quote above, because a foundation's total expenses reported on their tax forms may include non-charitable expenses (i.e. their own operating costs can count toward the 5% payout), we have to dig a little deeper in the 990 to find the actual amount paid out to charities. A form 990 can be a very dense document to sift through. The Guidestar API makes it much easier to find specific indicators of a charity's financial practices and overall health. To search for a specific charity, the only piece of information you need to provide is the organization's IRS-issued employer identification number, known as an EIN. 

To test this out and create a sampling of charitable giving rates, I looked up the EINs for some of the largest charitable foundations in the U.S.  

* Bill and Melinda Gates Foundation: 56-2618866
* Chan Zuckerberg Initiative Foundation: 45-5002209
* Bloomberg Family Foundation Inc (Bloomberg Philanthropies): 20-5602483
* The Larry Ellison Foundation: 94-3269827
* Sheryl Sandberg and Dave Goldberg Family Foundation: 46-1194887
* Carl Victor Page Memorial Foundation: 20-1922957
* Ford Foundation: 13-1684331
* Lilly Endowment: 35-0868122
* Foundation to Promote Open Society (George Soros): 26-3753801
* Robert Wood Johnson Foundation: 22-6029397
* Rockefeller Foundation: 13-1659629
* William and Flora Hewlett Foundation: 94-1655673

The people who run the first five foundations listed below have signed [The Giving Pledge](https://givingpledge.org/About.aspx), an initiative launched 11 years ago by Bill Gates and Warren Buffett urging fellow billionaires to make a public commitment  to give away a majority of their personal fortunes within their lifetimes. This is not a new idea: In 1889, philanthropist Andrew Carnegie wrote "The Gospel of Wealth," which called for those with exceptional resources to give away the majority of their wealth, during their lifetimes, to support community needs. Working in fundraising, I often wonder about the extent to which charitable foundations serve this goal, and whether or not they should. 


In [1]:
import json

def load_api_key():
    """reads in personal API key without printing the actual key for security"""
    api_key = " "
    with open(r"C:\Users\racha\Desktop\Python2\.gitignore\apik_sec.json", 'r') as sensitive_file:
        s = json.loads(sensitive_file.read())
        api_key = s['API_KEY']
    return api_key



In [2]:
def build_charity_url(ein):
    """takes in an employer ID number (EIN) and returns a url that can be used with Guidestar's API"""
    API_endpoint = 'https://apidata.guidestar.org/premier/v3/'
    ein_plus_key = "%s?Subscription-Key=%s" % (ein, load_api_key())
    full_url = API_endpoint + ein_plus_key
    return full_url

In [3]:
import json, requests

def get_charity_990(url):
    """sends a request to Guidestar's API for information on a given organization, then converts the response from 
    JSON to Python"""
    # send the request over the network and store response in resp
    resp = requests.get(url)
    # check for status code 200
    if(int(resp.status_code) == 200):
        # load API request response as Python
        profile_objects = json.loads(resp.text)
        return profile_objects
    else:
        return None
    

In [4]:
def calc_charity_payout(profile_objects):
    """digs through a given charity's form 990 and calculates the percentage of assets donated to charity in the 
    most recent tax year"""
    # peel back the layers of information on the organization's financial data
    overview = profile_objects['data']
    financials = overview['financials']
    recent = financials['most_recent_year_financials']
    grants = int(recent['expense_gifts_grants_paid'])
    print("Total grants and contributions made: ", grants)

    revenue = (recent['total_revenue'])
    print("Total revenue: ", revenue)

    assets = int(recent['assets_total'])
    print("Total assets: ", assets)
        
    # print 'expense_gifts_grants_paid' as a percentage of 'assets_total'
    assets_payout = grants / assets
    assets_payout = "{:.1%}".format(assets_payout)
    print("Percent of total assets paid to charity: ", assets_payout)
        
  

In [5]:
# Bill & Melinda Gates Foundation
calc_charity_payout(get_charity_990(build_charity_url('56-2618866')))

Total grants and contributions made:  4607760270
Total revenue:  None
Total assets:  47850466255
Percent of total assets paid to charity:  9.6%


What are giving rates like for some of our major local foundations?

In [7]:
# Richard King Mellon Foundation ($2.5 billion in assets)
calc_charity_payout(get_charity_990(build_charity_url('25-1127705')))

Total grants and contributions made:  102066792
Total revenue:  0.0
Total assets:  2453031404
Percent of total assets paid to charity:  4.2%


In [8]:
# The Heinz Endowments ($1.6 billion in assets)
calc_charity_payout(get_charity_990(build_charity_url('25-1721100')))

Total grants and contributions made:  53765366
Total revenue:  0.0
Total assets:  1634663959
Percent of total assets paid to charity:  3.3%


In [9]:
# Hillman Family Foundations 
calc_charity_payout(get_charity_990(build_charity_url('25-6065959')))

Total grants and contributions made:  41567640
Total revenue:  None
Total assets:  1397500779
Percent of total assets paid to charity:  3.0%


So far, only one foundation I've checked on through this API gave away more than 5% of its assets to charity in the most recent tax year: The Bill and Melinda Gates Foundation. What I'd like to do next is compile these giving percentages into a dataframe, find the mean and median annual giving percentage, and then create a visualization to map them out. 